# Detect Model Bias with Amazon SageMaker Clarify


## Amazon Science: _[How Clarify helps machine learning developers detect unintended bias](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)_ 

[<img src="img/amazon_science_clarify.png"  width="100%" align="left">](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)

# Terminology

* **Bias**: 
An imbalance in the training data or the prediction behavior of the model across different groups, such as age or income bracket. Biases can result from the data or algorithm used to train your model. For instance, if an ML model is trained primarily on data from middle-aged individuals, it may be less accurate when making predictions involving younger and older people.

* **Bias metric**: 
A function that returns numerical values indicating the level of a potential bias.

* **Bias report**:
A collection of bias metrics for a given dataset, or a combination of a dataset and a model.

* **Label**:
Feature that is the target for training a machine learning model. Referred to as the observed label or observed outcome.

* **Positive label values**:
Label values that are favorable to a demographic group observed in a sample. In other words, designates a sample as having a positive result.

* **Negative label values**:
Label values that are unfavorable to a demographic group observed in a sample. In other words, designates a sample as having a negative result.

* **Facet**:
A column or feature that contains the attributes with respect to which bias is measured.

* **Facet value**:
The feature values of attributes that bias might favor or disfavor.

# Posttraining Bias Metrics
https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html

* **Difference in Positive Proportions in Predicted Labels (DPPL)**:
Measures the difference in the proportion of positive predictions between the favored facet a and the disfavored facet d.

* **Disparate Impact (DI)**:
Measures the ratio of proportions of the predicted labels for the favored facet a and the disfavored facet d.

* **Difference in Conditional Acceptance (DCAcc)**:
Compares the observed labels to the labels predicted by a model and assesses whether this is the same across facets for predicted positive outcomes (acceptances).

* **Difference in Conditional Rejection (DCR)**:
Compares the observed labels to the labels predicted by a model and assesses whether this is the same across facets for negative outcomes (rejections).

* **Recall Difference (RD)**:
Compares the recall of the model for the favored and disfavored facets.

* **Difference in Acceptance Rates (DAR)**:
Measures the difference in the ratios of the observed positive outcomes (TP) to the predicted positives (TP + FP) between the favored and disfavored facets.

* **Difference in Rejection Rates (DRR)**:
Measures the difference in the ratios of the observed negative outcomes (TN) to the predicted negatives (TN + FN) between the disfavored and favored facets.

* **Accuracy Difference (AD)**:
Measures the difference between the prediction accuracy for the favored and disfavored facets.

* **Treatment Equality (TE)**:
Measures the difference in the ratio of false positives to false negatives between the favored and disfavored facets.

* **Conditional Demographic Disparity in Predicted Labels (CDDPL)**:
Measures the disparity of predicted labels between the facets as a whole, but also by subgroups.

* **Counterfactual Fliptest (FT)**:
Examines each member of facet d and assesses whether similar members of facet a have different model predictions.


In [1]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

In [2]:
%store -r role

In [3]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Test data for bias

We created test data in JSONLines format to match the model inputs. 

In [4]:
test_data_bias_path = "./data-clarify/test_data_bias.jsonl"

In [5]:
!head -n 1 $test_data_bias_path

{"features":["I have been using Quicken for years now and it does everything that I need it to accomplish for my personal finances.","Digital_Software"],"star_rating":4}


### Upload the data

In [6]:
test_data_bias_s3_uri = sess.upload_data(bucket=bucket, key_prefix="bias/test_data_bias", path=test_data_bias_path)
test_data_bias_s3_uri

's3://sagemaker-us-east-1-895677591816/bias/test_data_bias/test_data_bias.jsonl'

In [7]:
!aws s3 ls $test_data_bias_s3_uri

2022-08-18 21:21:08     197983 test_data_bias.jsonl


In [8]:
%store test_data_bias_s3_uri

Stored 'test_data_bias_s3_uri' (str)


# Run Posttraining Model Bias Analysis

In [9]:
%store -r pipeline_name

In [10]:
print(pipeline_name)

BERT-pipeline-1660855137


In [11]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

Succeeded
[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:895677591816:pipeline/bert-pipeline-1660855137/execution/kgxe5qy4c7q2',
  'PipelineExecutionDisplayName': 'execution-1660855145919',
  'PipelineExecutionStatus': 'Succeeded',
  'StartTime': datetime.datetime(2022, 8, 18, 20, 39, 5, 797000, tzinfo=tzlocal())}]
CPU times: user 12 ms, sys: 3.99 ms, total: 16 ms
Wall time: 265 ms


# List Pipeline Execution Steps


In [12]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

Succeeded


In [13]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:895677591816:pipeline/bert-pipeline-1660855137/execution/kgxe5qy4c7q2


In [14]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

{'PipelineExecutionSteps': [{'AttemptCount': 0,
                             'EndTime': datetime.datetime(2022, 8, 18, 21, 14, 3, 971000, tzinfo=tzlocal()),
                             'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:895677591816:model-package/bert-reviews-1660855139/1'}},
                             'StartTime': datetime.datetime(2022, 8, 18, 21, 14, 2, 371000, tzinfo=tzlocal()),
                             'StepName': 'RegisterModel',
                             'StepStatus': 'Succeeded'},
                            {'AttemptCount': 0,
                             'EndTime': datetime.datetime(2022, 8, 18, 21, 14, 3, 869000, tzinfo=tzlocal()),
                             'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:895677591816:model/pipelines-kgxe5qy4c7q2-createmodel-st3mwi6qvy'}},
                             'StartTime': datetime.datetime(2022, 8, 18, 21, 14, 2, 371000, tzinfo=tzlocal()),
                             'StepName'

# View Created Model
_Note:  If the trained model did not pass the Evaluation step (> accuracy threshold), it will not be created._

In [15]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

pipeline_model_name = model_arn.split("/")[-1]
print(pipeline_model_name)

arn:aws:sagemaker:us-east-1:895677591816:model/pipelines-kgxe5qy4c7q2-createmodel-st3mwi6qvy
pipelines-kgxe5qy4c7q2-createmodel-st3mwi6qvy


# SageMakerClarifyProcessor

In [16]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.c5.2xlarge", 
    sagemaker_session=sess
)

# Writing DataConfig and ModelConfig
A `DataConfig` object communicates some basic information about data I/O to Clarify. We specify where to find the input dataset, where to store the output, the target column (`label`), the header names, and the dataset type.

Similarly, the `ModelConfig` object communicates information about your trained model and `ModelPredictedLabelConfig` provides information on the format of your predictions.  

**Note**: To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. `ModelConfig` specifies your preferred instance type and instance count used to run your model on during Clarify's processing.

## DataConfig

In [17]:
bias_report_prefix = "bias/report-{}".format(pipeline_model_name)

bias_report_output_path = "s3://{}/{}".format(bucket, bias_report_prefix)

data_config = clarify.DataConfig(
    s3_data_input_path=test_data_bias_s3_uri,
    s3_output_path=bias_report_output_path,
    label="star_rating",
    features="features",
    # label must be last, features in exact order as passed into model
    headers=["review_body", "product_category", "star_rating"],
    dataset_type="application/jsonlines",
)

## ModelConfig

In [18]:
model_config = clarify.ModelConfig(
    model_name=pipeline_model_name,
    instance_type="ml.m5.4xlarge",
    instance_count=1,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    # {"features": ["the worst", "Digital_Software"]}
    content_template='{"features":$features}',
)

## _Note: `label` is set to the JSON key for the model prediction results_

In [19]:
predictions_config = clarify.ModelPredictedLabelConfig(label="predicted_label")

## BiasConfig

In [20]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[
        5,
        4,
    ],  # needs to be int or str for continuous dtype, needs to be >1 for categorical dtype
    facet_name="product_category",
)

# Run Clarify Job

In [21]:
clarify_processor.run_post_training_bias(
    data_config=data_config,
    data_bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    #    methods='all', # FlipTest requires all columns to be numeric
    methods=["DPPL", "DI", "DCA", "DCR", "RD", "DAR", "DRR", "AD", "TE"],
    wait=False,
    logs=False,
)


Job Name:  Clarify-Posttraining-Bias-2022-08-18-21-21-09-948
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-895677591816/bias/test_data_bias/test_data_bias.jsonl', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-895677591816/bias/report-pipelines-kgxe5qy4c7q2-createmodel-st3mwi6qvy/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-895677591816/bias/report-pipelines-kgxe5qy4c7q2-createmodel-st3mwi6qvy', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMo

In [22]:
run_post_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_post_training_bias_processing_job_name

'Clarify-Posttraining-Bias-2022-08-18-21-21-09-948'

In [23]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [24]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [25]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, bias_report_prefix
        )
    )
)

In [26]:
from pprint import pprint

running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=run_post_training_bias_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

pprint(processing_job_description)

{'AppSpecification': {'ImageUri': '205585389593.dkr.ecr.us-east-1.amazonaws.com/sagemaker-clarify-processing:1.0'},
 'CreationTime': datetime.datetime(2022, 8, 18, 21, 21, 10, 104000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 8, 18, 21, 21, 10, 104000, tzinfo=tzlocal()),
 'ProcessingInputs': [{'AppManaged': False,
                       'InputName': 'dataset',
                       'S3Input': {'LocalPath': '/opt/ml/processing/input/data',
                                   'S3CompressionType': 'None',
                                   'S3DataDistributionType': 'FullyReplicated',
                                   'S3DataType': 'S3Prefix',
                                   'S3InputMode': 'File',
                                   'S3Uri': 's3://sagemaker-us-east-1-895677591816/bias/test_data_bias/test_data_bias.jsonl'}},
                      {'AppManaged': False,
                       'InputName': 'analysis_config',
                       'S3Input': {'LocalPat

In [27]:
running_processor.wait(logs=False)

.........................................................*

UnexpectedStatusException: Error for Processing job Clarify-Posttraining-Bias-2022-08-18-21-21-09-948: Failed. Reason: ClientError: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'ml.m5.4xlarge for endpoint usage' is 1 Instances, with current utilization of 1 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.

# Download Report From S3

In [ ]:
!aws s3 ls $bias_report_output_path/

In [ ]:
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./generated_bias_report/report.html">Bias Report</a></b>'))

# View Bias Report in Studio
In Studio, you can view the results under the experiments tab.

<img src="img/bias_report.gif">

Each bias metric has detailed explanations with examples that you can explore.

<img src="img/bias_detail.gif">

You could also summarize the results in a handy table!

<img src="img/bias_report_chart.gif">

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>